In [15]:
## from tree_construct import *
# from likelihood_calculation import *
import glhap_app
from pysam import VariantFile, FastaFile
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [16]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [17]:
if is_notebook() == False:
    parser = argparse.ArgumentParser(description='Calculation of haplogroups')
    parser.add_argument('tree', type=str, help='haplogroup tree')
    parser.add_argument('ref', type=str, help='reference fasta')
    parser.add_argument('vcf', type=str, help='vcf file of mtdna')
    args = parser.parse_args()

    bcf_in = VariantFile(args.vcf) 
    ref = FastaFile(args.ref)
else:
    tree = 'array/array.json'
    ref = 'refchrm.fa'
    bcf_in = 'HGDP00900.alt_bwamem_GRCh38DH.20181023.Russian.cram.mpileup.vcf.gz'
    # bcf_in = 'in1.mpileup.vcf.gz'

In [14]:
# print(RenderTree(glhap_app.glhap(ref, tree, bcf_in)))
glhap_app.glhap(ref, tree, bcf_in)

Haplogroup     |	PL score       
H2a2b          |	-166.20000000000002
mt-MRCA (RSRS) |	-167.8         
H2a2b4         |	-186.00000000000003
H2a2b5         |	-188.10000000000002
L1'2'3'4'5'6   |	-188.20000000000002
L2'3'4'5'6     |	-188.20000000000002
H2a2b2         |	-188.8         
H2a2b3         |	-189.60000000000002



"Haplogroup     |\tPL score       \nH2a2b          |\t-166.20000000000002\nmt-MRCA (RSRS) |\t-167.8         \nH2a2b4         |\t-186.00000000000003\nH2a2b5         |\t-188.10000000000002\nL1'2'3'4'5'6   |\t-188.20000000000002\nL2'3'4'5'6     |\t-188.20000000000002\nH2a2b2         |\t-188.8         \nH2a2b3         |\t-189.60000000000002\n"

In [ ]:
def glhap_aposterior(bcf_in_str):
    with open('array/array.json') as f:
        d = json.load(f) # d - это список python
    for i in d:
        i[0] += 1
    d[0][0]=0

    bcf_in = VariantFile(bcf_in_str) 
    ref = FastaFile('refchrm.fa')
    a = Node(d[0][1],[])
    make_tree(d,a,0)
    ref_lh = calculate_likelihood(bcf_in, ref)
    gls = get_log_monozygous(bcf_in)
    prunung(a, ref, ref_lh, gls)
    key = lambda x: x.lh
    S = list()
    for i in PreOrderIter(a):
        S.append(i)
    S.sort(key = key,reverse=True)
    
    aposterior_divisor = sum([2**S[i].lh for i in range(len(S))])
    
    return [2**S[i].lh/aposterior_divisor for i in range(10)]
    


In [ ]:
make_tree(